In [2]:
# Install the astral library for celestial calculations
%pip install astral

In [3]:
# Install necessary libraries for the weather application
!pip install -q gradio requests plotly timezonefinder python-dateutil astral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00


In [4]:
# Install the prophet library for time series forecasting
!pip install prophet

In [5]:
# Install libraries for image conversion (SVG to PNG and PNG to GIF)
!pip install cairosvg pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00


In [8]:
import os
import cairosvg
from PIL import Image

# Define input and output directories for image conversion
INPUT_DIR = "/content"
OUTPUT_DIR = "/content/converted"

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def svg_to_png(svg_path, png_path, scale=256):
    """Convert SVG → PNG with default size if not defined."""
    try:
        # Use cairosvg to convert SVG to PNG
        cairosvg.svg2png(
            url=svg_path,
            write_to=png_path,
            output_width=scale,
            output_height=scale
        )
        print(f"✅ {svg_path} → {png_path}")
    except Exception as e:
        print(f"❌ Failed {svg_path}: {e}")

def png_to_gif(png_path, gif_path):
    """Convert PNG → GIF (single frame)."""
    try:
        # Use Pillow to convert PNG to GIF
        img = Image.open(png_path).convert("RGBA")
        img.save(gif_path, save_all=True, loop=0)
        print(f"✅ {png_path} → {gif_path}")
    except Exception as e:
        print(f"❌ Failed {png_path}: {e}")

# Loop through all SVG files in the input directory
for file in os.listdir(INPUT_DIR):
    if file.endswith(".svg"):
        name = os.path.splitext(file)[0]
        svg_path = os.path.join(INPUT_DIR, file)
        png_path = os.path.join(OUTPUT_DIR, f"{name}.png")
        gif_path = os.path.join(OUTPUT_DIR, f"{name}.gif")

        # Convert SVG → PNG (force 256x256)
        svg_to_png(svg_path, png_path, scale=256)

        # Convert PNG → GIF
        if os.path.exists(png_path):
            png_to_gif(png_path, gif_path)

print("🎉 All SVG files converted! Check /content/converted/")

✅ /content/snowy-4.svg → /content/converted/snowy-4.png
✅ /content/converted/snowy-4.png → /content/converted/snowy-4.gif
✅ /content/cloudy-night-1.svg → /content/converted/cloudy-night-1.png
✅ /content/converted/cloudy-night-1.png → /content/converted/cloudy-night-1.gif
✅ /content/rainy-1.svg → /content/converted/rainy-1.png
✅ /content/converted/rainy-1.png → /content/converted/rainy-1.gif
✅ /content/rainy-4.svg → /content/converted/rainy-4.png
✅ /content/converted/rainy-4.png → /content/converted/rainy-4.gif
✅ /content/cloudy.svg → /content/converted/cloudy.png
✅ /content/converted/cloudy.png → /content/converted/cloudy.gif
✅ /content/weather_sunset.svg → /content/converted/weather_sunset.png
✅ /content/converted/weather_sunset.png → /content/converted/weather_sunset.gif
✅ /content/snowy-1.svg → /content/converted/snowy-1.png
✅ /content/converted/snowy-1.png → /content/converted/snowy-1.gif
✅ /content/snowy-3.svg → /content/converted/snowy-3.png
✅ /content/converted/snowy-3.png → /co

In [9]:
# weather_advisor_app.py

# Install required packages if not already installed
# !pip install gradio requests geopy matplotlib scikit-learn

import requests
import gradio as gr
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime, timedelta

# -------------------------------
# Config
# -------------------------------
OPEN_METEO_URL = "https://api.open-meteo.com/v1/forecast"
ICONS_DIR = "/content/converted"  # <-- where your .gif files are stored

# Mapping of weather conditions to corresponding image file paths
condition_images = {
    "sunny": f"{ICONS_DIR}/day.gif",
    "cloudy": f"{ICONS_DIR}/cloudy.gif",
    "rain": f"{ICONS_DIR}/rainy-5.gif",
    "storm": f"{ICONS_DIR}/thunder.gif",
    "default": f"{ICONS_DIR}/default.gif"
}

# -------------------------------
# Utility functions
# -------------------------------
def get_coordinates(place: str):
    """Get geographical coordinates (latitude, longitude) for a given place name."""
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(place)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError("❌ Place not found. Try another name.")

def fetch_weather(lat, lon):
    """Fetch weather and air quality data for given coordinates."""
    forecast_params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True,
        "hourly": "temperature_2m,precipitation,weathercode",
        "timezone": "auto"
    }
    # Fetch weather data from Open-Meteo
    r1 = requests.get(OPEN_METEO_URL, params=forecast_params)
    data1 = r1.json()

    if "current_weather" not in data1:
        raise ValueError("Weather data not available.")

    weather = data1["current_weather"]
    hourly = data1.get("hourly", {})

    temp = weather.get("temperature", "N/A")
    wind = weather.get("windspeed", "N/A")
    precip = hourly.get("precipitation", ["N/A"])[0]
    weather_code = weather.get("weathercode", 0)

    # Air Quality API
    aq_url = "https://air-quality-api.open-meteo.com/v1/air-quality"
    aq_params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "pm10,uv_index"
    }
    # Fetch air quality data
    r2 = requests.get(aq_url, params=aq_params)
    data2 = r2.json()

    pm10 = data2.get("hourly", {}).get("pm10", ["N/A"])[0]
    uv = data2.get("hourly", {}).get("uv_index", ["N/A"])[0]

    # Return also real hourly temps
    hourly_times = hourly.get("time", [])
    hourly_temps = hourly.get("temperature_2m", [])

    return temp, wind, precip, pm10, uv, weather_code, hourly_times, hourly_temps

def get_weather_icon(weather_code):
    """Map Open-Meteo weather codes to GIFs."""
    if weather_code in [0, 1]:  # clear sky
        return condition_images["sunny"]
    elif weather_code in [2, 3]:  # cloudy
        return condition_images["cloudy"]
    elif weather_code in range(51, 68) or weather_code in range(80, 83):  # rain
        return condition_images["rain"]
    elif weather_code in range(95, 100):  # thunderstorm
        return condition_images["storm"]
    else:
        return condition_images["default"]


def generate_advice(temp, wind, precip, pm10, uv):
    """Generate smart advice messages based on weather and air quality data."""
    advice = []
    try:
        if temp != "N/A":
            if temp > 30:
                advice.append("🥵 It's hot! Stay hydrated.")
            elif temp < 10:
                advice.append("🧥 It's cold. Wear warm clothes.")

        if wind != "N/A" and wind > 20:
            advice.append("💨 Strong winds! Be careful outside.")

        if precip != "N/A" and precip > 0:
            advice.append("☔ Rain expected. Carry an umbrella.")

        if pm10 != "N/A" and pm10 > 50:
            advice.append("😷 Air quality is bad. Wear a mask.")

        if uv != "N/A" and uv > 6:
            advice.append("🕶️ UV is high. Use sunscreen.")

    except Exception:
        pass

    if not advice:
        advice = ["✅ Weather looks fine today!"]

    return " | ".join(advice)

from prophet import Prophet
import pandas as pd

def prophet_forecast(times, temps):
    """Generate a temperature forecast using the Prophet model."""
    if len(times) < 2:
        return None

    # Build DataFrame for Prophet
    df = pd.DataFrame({
        "ds": [datetime.fromisoformat(t) for t in times],
        "y": temps
    })

    # Initialize and fit the Prophet model
    model = Prophet(daily_seasonality=True)
    model.fit(df)

    # Make future predictions
    future = model.make_future_dataframe(periods=24, freq="H")
    forecast = model.predict(future)

    # Plot the forecast
    fig = model.plot(forecast)
    plt.title("Temperature Forecast (Prophet)")
    return fig


def smart_weather_place(place):
    """Main function to get weather data, generate advice, and provide forecast for a given place."""
    try:
        # Get coordinates for the place
        lat, lon = get_coordinates(place)

        # Fetch weather and air quality data
        (temp, wind, precip, pm10, uv, weather_code,
         hourly_times, hourly_temps) = fetch_weather(lat, lon)

        # Generate weather-based advice
        advice = generate_advice(temp, wind, precip, pm10, uv)
        # Get the appropriate weather icon
        icon = get_weather_icon(weather_code)
        # Get the current local time
        local_time = datetime.now().strftime("%Y-%m-%d %H:%M")

        # Use Prophet forecast with real hourly temps
        forecast_plot = prophet_forecast(hourly_times, hourly_temps) if hourly_temps else None

        # Format weather information
        weather_str = f"🌡 {temp}°C, 🌬 {wind} km/h, ☔ {precip} mm, PM10={pm10}, UV={uv}"
        return local_time, weather_str, advice, forecast_plot, icon

    except Exception as e:
        # Handle errors and return default values
        return None, f"Error: {str(e)}", "", None, condition_images["default"]


# -------------------------------
# Gradio App
# -------------------------------
# Define the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Smart Weather Advisor (Place-based)")

    # Input field for place name
    place = gr.Textbox(label="Enter Place (e.g. New York, London, Tokyo)", value="New York")
    # Button to trigger weather data retrieval
    btn = gr.Button("Get Weather & Forecast")

    # Output fields to display results
    with gr.Row():
        time_out = gr.Textbox(label="Local Time")
        weather_out = gr.Textbox(label="Weather, Air Quality & Precipitation")
        advice_out = gr.Textbox(label="Advice")

    with gr.Row():
        forecast_out = gr.Plot(label="ML Forecasts")
        image_out = gr.Image(label="Weather Illustration", type="filepath")

    # Connect the button click to the main function and specify inputs/outputs
    btn.click(smart_weather_place, [place],
              [time_out, weather_out, advice_out, forecast_out, image_out])

# Launch the Gradio app
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db780a0d6052ad9c46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
